In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensornetworks_pytorch.TNModels import PosMPS, Born
print(torch.__version__)

1.8.0.dev20201128


Use some data

In [2]:
import pickle
for dataset in [#'biofam',
    'flare','lymphography','spect','tumor','votes']:
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0].astype(int)
    print(dataset)
    print("\tdata shape:", X.shape)
    print(f"\trange of X values: {X.min()} -- {X.max()}")

def load_dataset(dataset):
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0]
    X=X.astype(int)

    print("data shape:", X.shape)
    print(f"range of X values: {X.min()} -- {X.max()}")
    print(f"setting d={X.max()+1}")
    d = X.max()+1
    return X, d

flare
	data shape: (1065, 13)
	range of X values: 0 -- 7
lymphography
	data shape: (148, 19)
	range of X values: 0 -- 7
spect
	data shape: (187, 23)
	range of X values: 0 -- 1
tumor
	data shape: (339, 17)
	range of X values: 0 -- 3
votes
	data shape: (435, 17)
	range of X values: 0 -- 2


In [11]:
X,d = load_dataset('lymphography')

data shape: (148, 19)
range of X values: 0 -- 7
setting d=8


In [12]:
D = 4
mps = PosMPS(dataset=X, D=D, d=d, homogeneous=False, verbose=True)
rBorn = Born(dataset=X, D=D, d=d, dtype=torch.float, homogeneous=False, verbose=True) 
cBorn = Born(dataset=X, D=D, d=d, dtype=torch.cfloat, verbose=True)
models = (mps, rBorn, cBorn)
for model in models:
    print(model.core.shape, model.name)

nonhomogeneous init
nonhomogeneous init
torch.Size([19, 8, 4, 4]) Positive MPS
torch.Size([19, 8, 4, 4]) Born model torch.float32
torch.Size([8, 4, 4]) Born model torch.complex64


In [13]:
cBorn.log_trick

True

In [14]:
from tqdm.notebook import tqdm
batchsize=20
trainloader = DataLoader(X, batch_size=batchsize, shuffle=True)
model = cBorn
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.5)

max_epochs = 20
print("Training...")
av_batch_loss_running = -1000
for epoch in tqdm(range(max_epochs)):
    print("epoch", epoch)
    batch_loss = []
    for batch_idx, batch in enumerate(trainloader):
        if torch.isnan(model.core).any():
            print("NANS!")
            break
        model.zero_grad()
        neglogprob = 0
        for x in batch:
            out = model(x)
            print(out)
            neglogprob -= out
#             print("out",out)
        loss = neglogprob / len(batch)
#         print("core befor\n",model.core[0][0])
#         if batch_idx>0:
#             print("\tgrad:",model.core.grad[0,0])
        loss.backward()
#         print("backward.")
#         print("\tgrad:",model.core.grad[0,0])
        optimizer.step()
#         print("optimize\n",model.core[0][0])
#         print("\tgrad:",model.core.grad[0,0])
        with torch.no_grad():
            batch_loss.append(loss.item())
            n=10
            if batch_idx % n == 0: # print every nth batch loss
                print("\tbatch", batch_idx, "loss", loss.item())
    av_batch_loss = torch.Tensor(batch_loss).mean().item()
    print("\tavg batch_loss", av_batch_loss)
    if abs(av_batch_loss_running - av_batch_loss) < .0001:
        print("Early stopping")
        break
    av_batch_loss_running = av_batch_loss
print('Finished training. Last av loss = ', av_batch_loss)

Training...


epoch 0
tensor(-37.1143, grad_fn=<SubBackward0>)
tensor(-37.7334, grad_fn=<SubBackward0>)
tensor(-37.8926, grad_fn=<SubBackward0>)
tensor(-37.4124, grad_fn=<SubBackward0>)
tensor(-38.1490, grad_fn=<SubBackward0>)
tensor(-37.1473, grad_fn=<SubBackward0>)
tensor(-37.7668, grad_fn=<SubBackward0>)
tensor(-37.9160, grad_fn=<SubBackward0>)
tensor(-38.5696, grad_fn=<SubBackward0>)
tensor(-36.9758, grad_fn=<SubBackward0>)
tensor(-38.3050, grad_fn=<SubBackward0>)
tensor(-37.7094, grad_fn=<SubBackward0>)
tensor(-37.3197, grad_fn=<SubBackward0>)
tensor(-37.2873, grad_fn=<SubBackward0>)
tensor(-38.1722, grad_fn=<SubBackward0>)
tensor(-37.0764, grad_fn=<SubBackward0>)
tensor(-37.5780, grad_fn=<SubBackward0>)
tensor(-37.7856, grad_fn=<SubBackward0>)
tensor(-37.0055, grad_fn=<SubBackward0>)
tensor(-37.3985, grad_fn=<SubBackward0>)
	batch 0 loss 37.61574935913086
NANS!
	avg batch_loss 37.61574935913086
epoch 1
NANS!
	avg batch_loss nan
epoch 2
NANS!
	avg batch_loss nan
epoch 3
NANS!
	avg batch_loss na

In [10]:
model.core.grad[0,0]

tensor([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [25]:
real = torch.tensor([[1, 2],[1, 2]], dtype=torch.float32)
imag = torch.tensor([[3, 4],[1, 2]], dtype=torch.float32)
z = torch.complex(real, imag)
z.requires_grad=True
b = (2.7j*z + 3.4*z + 18)
L = b.mean().abs().square()
L.backward()
print(L.grad_fn)
print(z.grad.conj())

tensor([[44.7375+0.7375j, 44.7375+0.7375j],
        [44.7375+0.7375j, 44.7375+0.7375j]])


In [21]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
L = (a * a).sum()
print(L.grad_fn, a.grad_fn)
L.backward()
print(a.grad)

False
True
<SumBackward0 object at 0x7ff54f8b2c10> None
tensor([[-0.8076,  3.6026],
        [ 3.4140,  1.9627]])


## Log norm trick

For left boundary,
- store direction and lognorm

In [28]:
print(z)

def matnorm(mat):
    return torch.max(torch.sum(abs(mat), dim=1))

norm = matnorm
norm(z)

tensor([[1.+3.j, 2.+4.j],
        [1.+1.j, 2.+2.j]], requires_grad=True)


tensor(7.6344, grad_fn=<MaxBackward1>)